In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.patches as mpatches

# Initialize empty lists to store all CSV file paths and date objects
file_paths_north = []
file_paths_south = []
date_objects_north = []
date_objects_south = []

# Define the year ranges and directories for North and South
directories = {
    'north': (list(range(2010, 2022)) + [2022, 2023], '/efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics', '_09', file_paths_north, date_objects_north),
    'south': (range(2011, 2023), '/efs/jrclark/PFe Data/PFe_Catalogs/S PFe Characteristics', '_03', file_paths_south, date_objects_south),
}

# Function to populate file paths and date objects
def populate_file_paths_and_dates(years, directory, year_suffix, file_paths, date_objects):
    for year in years:
        yr = str(year)
        folder_path = os.path.join(directory, f'{yr}{year_suffix}')
        
        if os.path.exists(folder_path):
            files = os.listdir(folder_path)
            for file in files:
                if file.endswith('.csv'):
                    full_path = os.path.join(folder_path, file)
                    subfilename = file[12:25]
                    date_time_obj = datetime.strptime(subfilename, '%Y%m%dT%H%M')
                    file_paths.append(full_path)
                    date_objects.append(date_time_obj)
        else:
            print(f"Directory {folder_path} does not exist")

# Populate file paths and dates for North and South
populate_file_paths_and_dates(*directories['north'])
populate_file_paths_and_dates(*directories['south'])

# Sort the file paths and date objects
file_paths_north.sort()
date_objects_north.sort()
file_paths_south.sort()
date_objects_south.sort()

# Define the function to plot and save the images
def plot_and_save_images(file_paths, date_objects, latitude1, latitude2, save_folder):
    for i, x in enumerate(file_paths):
        faculaeframe = pd.read_csv(x)
        
        # Ensure required columns are present
        if not all(col in faculaeframe.columns for col in ['Lat', 'Lon', 'Area', 'Avg_Br']):
            print(f"Missing required columns in file: {x}")
            continue
        
        extracted_coords = faculaeframe.sort_values(by='Area', ascending=False)
        data_cond = extracted_coords[['X_Centroid', 'Y_Centroid', 'Lat', 'Lon', 'Area', 'Avg_Br']]
        all_areas_list = data_cond['Area'].tolist()
        all_bfield_list = data_cond['Avg_Br'].tolist()
        
        marker_size = [10 if j <= 10 else 15 if j <= 20 else 20 if j <= 30 else 25 for j in all_areas_list]
        
        color_symbol = []
        for k in all_bfield_list:
            if k <= -400:
                b = 'b'
            elif k <= -200:
                b = '#17BECF'
            elif k <= 0:
                b = '#9EDAE5'
            elif k <= 200:
                b = '#dede00'
            elif k <= 400:
                b = '#ff7f00'
            else:
                b = 'r'
            color_symbol.append(b)
        
        lon_list = data_cond['Lon'].tolist()
        lat_list = data_cond['Lat'].tolist()

        longitude = list(range(0, 361))
        long_arry = np.array(longitude)
        lat_arr1 = np.array(latitude1)
        lat_arr2 = np.array(latitude2)
        
        def toxy_coord(long_ln, lat_ln):
            theta = long_ln * (math.pi / 180.0)
            lat_in_rad = lat_ln * (math.pi / 180.0)
            R = 1
            rho = R * ((math.pi / 2) - abs(lat_in_rad))
            x_long = rho * np.sin(theta)
            y_lat = -rho * np.cos(theta)
            return x_long, y_lat

        x_70, y_70 = toxy_coord(long_arry, lat_arr1)
        x_80, y_80 = toxy_coord(long_arry, lat_arr2)
        x_arr, y_arr = toxy_coord(np.array(lon_list), np.array(lat_list))

        # Initialize plot
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.plot(x_70, y_70, color='b')
        ax.plot(x_80, y_80, color='r')
        ax.axhline(0, color='black', linewidth=1)
        ax.axvline(0, color='black', linewidth=1)

        scatter = ax.scatter(x_arr, y_arr, marker_size, color_symbol, marker='o')

        size_labels = ['A = (0, 10]', 'A = (10, 20]', 'A = (20, 30]', 'A > 30']
        size_handles, _ = scatter.legend_elements(prop="sizes", alpha=0.6)

        color_labels = ['B <=-400', 'B = (-400, -200]', 'B = (-200, 0]', 
                        'B = (0, 200]', 'B = (200, 400]', 'B > 400']
        color_symbolS = ['blue', '#17BECF','#9EDAE5','#dede00', '#ff7f00', 'r']
        # Create custom color handles
        color_handles = [mpatches.Patch(color=color_symbolS[i], label=color_labels[i]) for i in range(len(color_labels))]

        # Add color and size legend to the plot
        legend1 = ax.legend(color_handles, color_labels, loc="upper left", title="Colors", fontsize=10)
        legend2 = ax.legend(size_handles, size_labels, loc="upper right", title="Sizes", fontsize= 10)
        ax.add_artist(legend1)
        ax.add_artist(legend2)
        
        ax.plot(0, 0, '+', color='#FF7F0E', markersize=12)
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)

        title_date = date_objects[i].strftime('%Y-%m-%d %H:%M:%S')
        plt.title(f'Azimuthal Equidistant Projection of PFe {title_date}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.axis('equal')
        plt.tight_layout()

        plt.text(0.02, 0.32, f'{latitude1[0]}°')
        plt.text(0.02, 0.15, f'{latitude2[0]}°')

        ax = plt.gca()
        ax.grid(False, which='both')

        plt.savefig(os.path.join(save_folder, f'Frame_{title_date}.png'), bbox_inches='tight')
        plt.close()

# Define latitude bands and save directories
latitude_bands = {
    'north': ([70] * 361, [80] * 361, '/efs/okome7/movie_images/2010-2022_September_PFe'),
    'south': ([-70] * 361, [-80] * 361, '/efs/okome7/movie_images/2011-2022_March_PFe'),
}

# Plot and save images for both North and South
plot_and_save_images(file_paths_north, date_objects_north, *latitude_bands['north'])
plot_and_save_images(file_paths_south, date_objects_south, *latitude_bands['south'])


Directory /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2023_09 does not exist
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210920T1224.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210920T1424.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210920T1624.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210920T2124.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210921T0224.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210921T0424.csv
Missing required columns in file: /efs/jrclark/PFe Data/PFe_Catalogs/N PFe Characteristics/2021_09/PFe_Catalog_20210921T1424.csv
Missing

In [5]:
import os
import zipfile
from IPython.display import FileLink

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

folder_to_zip = '/efs/okome7/movie_images/2010-2022_September_PFe'  # Replace with your folder name
output_zip = '2010-2022_Sept_PFe.zip'  # Replace with your desired output zip file name

folder_to_zip2 = '/efs/okome7/movie_images/2011-2022_March_PFe'  # Replace with your folder name
output_zip2 = '2010-2022_Sept_PFe.zip'  # Replace with your desired output zip file name

zip_folder(folder_to_zip, output_zip)
zip_folder(folder_to_zip2, output_zip2)

FileLink(output_zip)
FileLink(output_zip2)

/home/jovyan/efs/okome7/Final North_South pynb/2010-2022_Sept_PFe.zip